## Notebook Purpose

Quantitative evaluation of qdess recon experiments

In [1]:
import os, sys
from os import listdir
from os.path import isfile, join
from matplotlib import pyplot as plt
import numpy as np
import torch

sys.path.append('/home/vanveen/ConvDecoder/')
from utils.evaluate import calc_metrics, normalize_img

In [2]:
def plot_list(arr_list, clim=None):
    
    # stretch_factor = 0.8/(160/512) = 2.56 
    # 0.8 = slice thickness, 160 = FOV in phase encode direction and 512 = number of kspace samples in the phase direction
    STRETCH_FACTOR = 2.56 
    
    NUM_COLS = len(arr_list)
    title_list = ['ground-truth', 'recon, 4x', 'recon, 6x', 'recon, 8x']
    
    fig = plt.figure(figsize=(40,40))
    
    for idx in range(NUM_COLS):
        ax = fig.add_subplot(1,NUM_COLS,idx+1)
        ax.imshow(arr_list[idx], cmap='gray', clim=clim, aspect=(1./STRETCH_FACTOR))
        ax.set_title(title_list[idx], fontsize=20)
        ax.axis('off')

### define test set

based off `/bmrNAS/people/arjun/data/stanford-qDESS/annotations/v0.0.2/splits.xlsx`, which has pathology-relevant samples `MTR_065` and `MTR_066` in its test set

In [9]:
path = '/bmrNAS/people/arjun/data/qdess_knee_2020/files_recon_calib-16/'

mtr_list = ['065', '066', '240', '156', '188', '199', '224', '198', '219', '096', '052', '196', '221', '241', '223', '178', '227', '099', '218', '248', '006', '005', '173', '048', '158', '080', '034', '144', '176', '244', '120', '235', '237', '030']
fn_list = ['MTR_{}.h5'.format(f) for f in mtr_list]

### slices of clinical relevance

In [ ]:
idx_kx_dict = {'MTR_065.h5': [219, 220, 221, 372, 373, 374],
               'MTR_066.h5': [277, 278, 279, 280]}

### compute metrics

In [5]:
ACCEL_LIST = [4]
ECHO_NUM = 1

NUM_VARS = len(ACCEL_LIST) + 1 #

im_list = np.empty((len(files_gt),NUM_VARS,512,160))

for idx_a, ACCEL in enumerate(ACCEL_LIST):
    
    path = '/bmrNAS/people/dvv/out_qdess/accel_{}x/echo{}/'.format(ACCEL, ECHO_NUM)
    files = [f for f in listdir(path) if isfile(join(path, f))]
    
    # filter
    
    files_gt = [f for f in files if '_gt.npy' in f]
    NUM_SAMPS = len(files_gt)

    ssim_list, psnr_list = [], []

    for idx_f, f in enumerate(files_gt):
        
        idx_ = idx_f + idx_a*NUM_SAMPS

        img_gt = np.load(path + f)
        img_dc = np.load(path + f.split('_gt.npy')[0] + '_dc.npy')
        
        # save to im_list - wonky order b/c want diff accels of same sample adjacent
        # vs. calculating psnr/ssim want diff samples of same accels adjacent
        if idx_a == 0: # save img_gt to position 0
            im_list[idx_f, idx_a] = img_gt
        im_list[idx_f, idx_a+1] = img_dc # save img_dc=x_accel to position 1,2,3

        _, _, ssim_, psnr_ = calc_metrics(img_dc, img_gt)
        ssim_list.append(ssim_), psnr_list.append(psnr_)

    ssim_list, psnr_list = np.asarray(ssim_list), np.asarray(psnr_list)
    

    print('\n {}x accel'.format(ACCEL))
    print('ssim ~N({}, {})'.format(np.round(ssim_list.mean(), 4), \
                                   np.round(ssim_list.std(), 4)))
    print('psnr ~N({}, {})'.format(np.round(psnr_list.mean(), 4), \
                                   np.round(psnr_list.std(), 4)))
im_list = np.asarray(im_list)


 4x accel
ssim ~N(0.8662, 0.0237)
psnr ~N(32.8236, 1.257)


### echo2 results
4x accel
ssim ~N(0.772, 0.0535)
psnr ~N(32.8001, 1.487)

 6x accel
ssim ~N(0.7339, 0.0537)
psnr ~N(31.8721, 1.514)

 8x accel
ssim ~N(0.7166, 0.0505)
psnr ~N(31.4235, 1.1096)

### cast all range of pixel values to be on [0,1]

network output has .mean() ~2, while gt has .mean() ~20000

In [6]:
im_list_norm = np.empty(im_list.shape)

for idx_s, samp in enumerate(im_list): # for each sample    
    for idx_v, var in enumerate(samp): # for each variation of that sample
        
        im_list_norm[idx_s, idx_v] = (var - var.min()) * (1. / var.max())

### apply stretch factor to images

- 0.8/(160/512) = 2.56 stretch factor
    - 0.8 = slice thickness, 160 = FOV in phase encode direction and 512 = number of kspace samples in the phase direction
        - FOV: distance over which an MR image is acquired or displayed
- qdess data in shape [nc, kx, ky, kz] = [16, 512, 512, 160] 
    - take central slice in kx. then undersample in [ky, kz]
    - in-plane voxels [kx, ky] are taken at 0.3mm x 0.3mm and thru-plane voxels are taken at spacing of 0.8mm
        - here "in-plane" refers to the sagittal plane, even though we are sampling in [ky,kz] and hence viewing axial images

if we want to view each true voxel equally...

### TODO: reason through this using FOV and k-space notes in g doc

### make difference maps w gt

In [7]:
im_diff_list = np.empty(im_list.shape)

for idx_s, samp in enumerate(im_list_norm): # for each sample
    
    gt = samp[0]

    for idx_v, var in enumerate(samp): # for each variation of that sample

        if idx_v == 0:
            im_diff = np.zeros(gt.shape)
        else:
            var_norm = normalize_img(gt, var) # normalizes arg 2
            im_diff = np.abs(var_norm - gt)    
        
        im_diff_list[idx_s, idx_v]  = im_diff

In [12]:
# for idx in np.arange(NUM_SAMPS):
#     plot_list(im_list_norm[idx])
#     plot_list(im_diff_list[idx], clim=(0,im_diff_list[idx].max()))